In [7]:
import pymongo
import requests
import json
import time
import re
from config import api_key

In [8]:
headers = {'Authorization': 'Bearer %s' % api_key}

In [9]:
# connecting to database
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.restaurant_db

In [10]:
# querying all records
restaurants = db.restaurant_atomic.find()

In [11]:
# convert phone number to format compatible with Yelp's API
def convert_phone(phone):
    phone = re.sub("[(]", "+1",phone)
    phone = re.sub("[)-]","",phone)
    phone = phone.replace(" ","")
    return phone

def get_match_data(id_number):
    # retrieve business information
    time.sleep(1)
    details_response = requests.get(f"https://api.yelp.com/v3/businesses/{id_number}", headers=headers).json()
    try:
        restaurant_name = details_response["name"]
    except:
        restaurant_name = name
    try:
        categories = details_response["categories"]
    except:
        categories = "N/A"
    try:
        coordinates = details_response["coordinates"]
    except:
        coordinates = "N/A"
    try:
        photos = details_response["photos"]
    except:
        photos = "N/A"
    try:
        location = details_response["location"]
    except KeyError:
        location = "N/A"
    try:
        price = details_response["price"]
    except KeyError:
        price = "N/A"
    try:
        hours = details_response["hours"]
    except KeyError:
        hours = "N/A"
    try:
        alias = details_response["alias"]
    except KeyError:
        alias = "N/A"
    try:
        image_url = details_response["image_url"]
    except KeyError:
        image_url = "N/A"
    try:
        is_closed = details_response["is_closed"]
    except KeyError:
        is_closed = "N/A"
    try:
        rating = details_response["rating"]
    except KeyError:
        rating = "N/A"
    try:
        review_count = details_response["review_count"]
    except KeyError:
        review_count = "N/A"
    try:
        transactions = details_response["transactions"]
    except KeyError:
        transactions = "N/A"
    try:
        yelp_page_url = details_response["url"]
    except KeyError:
        yelp_page_url = "N/A"
    try:
        store_phone = details_response["phone"]
    except KeyError:
        store_phone = "N/A"
    try:
        store_display_phone = details_response["display_phone"]
    except KeyError:
        store_display_phone = "N/A"
    try:
        is_claimed = details_response["is_claimed"]
    except KeyError:
        is_claimed = "N/A"

    # retrieve reviews of business
    reviews_response = requests.get(f"https://api.yelp.com/v3/businesses/{id_number}/reviews", headers=headers)
    time.sleep(1)
    try:
        reviews = reviews_response.json()["reviews"]
    except KeyError:
        reviews = "N/A"

    # create object to insert into database
    restaurant_dict = {
        "restaurant_name": restaurant_name,
        "alias": alias,
        "restaurant_phone": store_phone,
        "restaurant_display_phone": store_display_phone,
        "is_claimed": is_claimed,
        "is_closed": is_closed,
        "categories": categories,
        "price": price,
        "hours": hours,
        "transactions": transactions,
        "image_url": image_url,
        "pictures": photos,
        "location": location,
        "coordinates": coordinates,
        "rating": rating,
        "review_count": review_count,
        "reviews": reviews,
        "yelp_page_url": yelp_page_url
    }
    
    # check for duplicates records in db
    try:
        if db.yelp.find({"restaurant_name":restaurant_name, "location.address1":location["address1"]})[0]:
            print(f"Duplicate found by name and address: {restaurant_name}")
    except IndexError:
        # insert object into database
        db.yelp.insert_one(restaurant_dict) 
        print(f"Sucessful new record")


In [12]:
record_count = 0
for restaurant in restaurants[60860:]:
    name = restaurant["name"]
    address1 = restaurant["street_address"]
    city = restaurant["city"]
    state = "CO"
    country = "US"
    phone = restaurant["phone"]
    # request Yelp's Business Match API
    match_base_url = "https://api.yelp.com/v3/businesses/matches?"
    match_response = requests.get(f"{match_base_url}name={name}&address1={address1}&city={city}&state={state}&country={country}", headers=headers)
    time.sleep(1)
    # if a match is found, extract information
    try:
        if match_response.json()["businesses"]:
            # get Yelp's unique business ID number
            id_number = match_response.json()["businesses"][0]["id"]
            # use unique ID to extract information
            try:
                get_match_data(id_number)
            except:
                print("No match found")
        # if no match is found, search by phone number
        else:
            # if restaurant_db phone number is in correct format, continue
            if len(restaurant["phone"]) == 14:
                # convert phone number to format compatible with Yelp's Phone Search API
                phone = convert_phone(phone)
                # check for duplicates records in db
                try:
                    if db.yelp.find({"restaurant_phone":phone})[0]:
                        print(f"Duplicate found by phone: {phone}")
                        record_count += 1
                        print(f"Record count: {record_count}")
                        continue
                except IndexError:
                    pass
                # request Yelp's Phone Search API
                phone_base_url = "https://api.yelp.com/v3/businesses/search/phone?"
                phone_response = requests.get(f"{phone_base_url}phone={phone}", headers=headers)
                time.sleep(1)
                if phone_response.json()["businesses"]:
                    # get Yelp's unique business ID number
                    id_number = phone_response.json()["businesses"][0]["id"]
                    # use unique ID to extract information
                    try:
                        get_match_data(id_number)
                    except:
                        print("No match found")
                else:
                    print(f"No match found")
            else:
                print(f"No match found")
    except KeyError:
        print(f"No match found")
        
    record_count += 1
    print(f"Record count: {record_count}")

Sucessful new record
Record count: 1
No match found
Record count: 2
No match found
Record count: 3
No match found
Record count: 4
No match found
Record count: 5
Sucessful new record
Record count: 6
No match found
Record count: 7
Sucessful new record
Record count: 8
Sucessful new record
Record count: 9
Sucessful new record
Record count: 10
Duplicate found by name and address: Louies Pizza
Record count: 11
Duplicate found by name and address: Louie's Pizza
Record count: 12
Duplicate found by name and address: Louie's Pizza
Record count: 13
Sucessful new record
Record count: 14
Sucessful new record
Record count: 15
Sucessful new record
Record count: 16
Duplicate found by name and address: Lucky Dragon Chinese Restaurant
Record count: 17
Sucessful new record
Record count: 18
Sucessful new record
Record count: 19
Sucessful new record
Record count: 20
Sucessful new record
Record count: 21
Sucessful new record
Record count: 22
Duplicate found by name and address: Maggie Mae's Restaurant & Pub

Duplicate found by name and address: Oliver's Deli
Record count: 182
No match found
Record count: 183
Duplicate found by name and address: On The Border Mexican Grill & Cantina
Record count: 184
Duplicate found by name and address: On The Border Mexican Grill & Cantina
Record count: 185
Duplicate found by name and address: On The Border Mexican Grill & Cantina
Record count: 186
Duplicate found by name and address: On The Boulevard
Record count: 187
No match found
Record count: 188
No match found
Record count: 189
Duplicate found by name and address: North End Diner
Record count: 190
Duplicate found by name and address: Nosh
Record count: 191
Duplicate found by name and address: NYPD Pizza
Record count: 192
Duplicate found by name and address: O'furry's
Record count: 193
Duplicate found by name and address: Ola Juice Bar
Record count: 194
Duplicate found by name and address: Old Chicago Pizza & Taproom
Record count: 195
Duplicate found by name and address: Old Chicago Pizza & Taproom
Re

In [31]:
count = 0
for i in db.restaurant_atomic.find()[60860:]:
    count += 1
    print(f"{count} {i['name']} - {i['phone']}")

1 Orange Julius - (719) 598-4211
2 Orange Julius - (719) 597-6606
3 Oscar's Pub & Kitchen - (719) 597-3369
4 Oscars Tejon Street - (719) 471-8070
5 Outback Steakhouse - (205) 759-9000
6 Outback Steakhouse - 
7 Outback Steakhouse - (719) 590-6283
8 Overseas 101 - (719) 534-9588
9 Overtime Sports Bar & Grill - 
10 Overtimelounge - (719) 265-5294
11 Page's Deli Cafe - 
12 Page's Deli Cafe - (719) 528-6655
13 Palmer Lake Brewing Co - (719) 475-8880
14 Palmer's Restaurant - (719) 955-5600
15 Panda Express - 
16 Panda Express - 
17 Panda Express - (760) 864-8213
18 Panda Express - (719) 574-3126
19 Panda Express - (760) 864-8213
20 Panda Express - (719) 574-3126
21 Panda Garden Chinese Restaurant - (719) 574-2828
22 Panera Bread - 
23 Panera Bread - 
24 Panera Bread - 
25 Panino's Restaurant - (719) 886-0062
26 Panino's Restaurant - (719) 635-1188
27 Panino's Restaurant - (719) 635-1187
28 Paolas Mexican Food - (719) 471-1116
29 Papa Jo's Inc - (719) 633-8212
30 Papa John''s Pizza - (616) 82

1436 Dragon Gate - (303) 238-9922
1437 Dragon House - (303) 429-3841
1438 Duffey's Bakery & Cafe - (303) 753-9177
1439 Duffey's Bakery Cafe - (303) 778-9026
1440 Duffy's Beverage Co - 
1441 Duffy's Shamrock Restaurant & Bar - (303) 534-4935
1442 Dugout Italian Deli - (303) 298-1715
1443 Duo - (303) 477-4141
1444 Duo Restaurant - 
1445 Durango's N Restaurant - 
1446 Duy Restaurant - 
1447 Dyson's Cakes & More - (303) 394-1191
1448 Earl Of Sandwich - (303) 832-7411
1449 Earl's Restaurant & Bar - (303) 595-3275
1450 Earl's Restaurant Cherry Creek - 
1451 Earl's Restaurant Downtown - 
1452 Earthgrain Breads - (303) 428-1582
1453 Earthgrains Baking Companies - (303) 320-0407
1454 East Asia Garden - (720) 259-7141
1455 East Buffet Chinese Restaurant - (303) 935-3388
1456 East Side Kosher Deli - (303) 322-9862
1457 Eastern Bakery - (303) 853-0232
1458 Eat Dessert First - (303) 282-4954
1459 Eclipse Restaurant - 
1460 Edge Restaurant - 
1461 Edgewater Inn - (303) 237-3524
1462 Effin's Bar & Gr

2936 Sandwich Board - (303) 297-3046
2937 Sandwich Board The - 
2938 Sandwich Board The - 
2939 Sandwich Board The - 
2940 Santa Fe Cookie Co. - (303) 623-0919
2941 Santiago Mexican Resturant #9 - (303) 936-3030
2942 Santiago's Express - (303) 255-7945
2943 Santiago's Xiv Mexican Restaurant - 
2944 Santiagos Mex Restaurant - 
2945 Santiagos Mexican Restaurant - (303) 333-5305
2946 Santiagos Mexican Restaurant - 
2947 Santiagos Mexican Restaurant - (303) 430-9704
2948 Santiagos Mexican Restraunt - (303) 534-5004
2949 Santino's - (303) 733-7193
2950 Sara Lee Sandwich Shop - (303) 342-3372
2951 Satire Restaurant & Lounge - (303) 322-2227
2952 Saucy Noodle Ristorante - (303) 733-7497
2953 Saverino - (303) 308-0764
2954 Sawyers Subs - (303) 469-0911
2955 Sbarro - (205) 556-9811
2956 Sbarro - (303) 825-4807
2957 Schlotzsky's Deli - (303) 757-3711
2958 Schlotzsky's Deli - (303) 696-1100
2959 Schlotzsky's Deli - (303) 985-5555
2960 Schlotzsky's Deli Inc - (303) 453-0608
2961 Scoreboard Restaur

4435 Front Range Refrigeration Inc - (970) 204-1145
4436 Front Range Restaurant Association - (970) 266-2612
4437 Garbanzo Mediterranean Grill - (970) 449-5020
4438 Gib's New York Bagels - (970) 224-5946
4439 Gibs New York Bagel & Deli - (970) 223-5253
4440 Golden Corral - (970) 204-0395
4441 Golden Dragon Restaurant - (970) 493-4278
4442 Gonzalo's Spanish Manor - (970) 226-5986
4443 Good Times Drive Thru - (970) 221-1922
4444 Grillers Global Cafe - (970) 221-3077
4445 Hat Trixx - (970) 229-1599
4446 Hibachi Japanese Steakhouse And Sushi Bar - (970) 377-0812
4447 Hog Wild Bbq Inc - (970) 493-6029
4448 Home For Dinner - (970) 416-9378
4449 Hong Kong Buffet - (970) 221-5664
4450 Hooters - (770) 736-6560
4451 Hooters Restaurant - (970) 204-1244
4452 Hospitality Supply Inc - (970) 266-9555
4453 Hot Wok Cafe - (970) 491-9681
4454 Hu Hot Mongolian Grill - 
4455 Huhot Mongolian Grill - (970) 416-0555
4456 Hunan Chinese Restaurant - (970) 207-9000
4457 Ihop - (201) 617-5164
4458 Inca Mexican R

5935 Le Peep - (858) 679-2701
5936 Le Peep Restaurants - (303) 730-7105
5937 Leonard - 
5938 Lepeep - (303) 779-5908
5939 Links Golf Course Grill - (303) 470-1587
5940 Little Holly's Asia Cafe - (303) 683-5558
5941 Little King - (303) 948-3802
5942 Little Yen King - (303) 979-9555
5943 Lloyd's Barbeque Co - (303) 734-1326
5944 Lodo's Bar & Grill - (303) 346-2930
5945 Lone Star Steakhouse & Saloon - (303) 932-1718
5946 Los Dos Potrillos - (720) 529-0299
5947 Luigi''s Bent Noodle - (303) 694-1710
5948 Lunch Break 2 - (720) 493-1545
5949 Lunchbox The At Collumbine Valley - (303) 738-8784
5950 Lynnette Bash Cakes - (303) 973-2194
5951 M C's Pizza - (303) 763-8299
5952 Macaroni Grill - (732) 706-9533
5953 Mama Carmela's - (303) 716-1616
5954 Manna Bakery & Deli - (303) 798-7117
5955 Manor House Restaurant - (303) 973-8064
5956 Marie Callender's - (303) 795-8833
5957 Master Chef - (303) 730-7878
5958 Mc Donald''s - (330) 535-1441
5959 Mc Donald''s - (330) 535-1441
5960 Mc Donald's - (303) 47

6934 Mc Donald''s - (719) 775-2782
6935 Mcdonald's - (719) 488-0044
6936 Monument Pizza Company - (719) 488-2100
6937 Natural Grocers By Vitamin - (719) 487-0448
6938 Pizza Hut - (719) 481-4800
6939 Pure Delights Cafe - (719) 481-8558
6940 Restaurant Management Service - (719) 487-7287
6941 Rosie's Diner - (719) 481-3287
6942 Starbucks - (719) 488-8872
6943 Subway Sandwiches & Salads - (719) 481-8070
6944 Taco Bell - (719) 488-0190
6945 The Sand Trap Grill - (719) 488-0311
6946 Village Inn - (719) 481-3500
6947 Aramark Sports & Entertainment At Red Rocks - (303) 697-4939
6948 Beso De Arte - (303) 697-3377
6949 Big Yellow House - (303) 697-7564
6950 Blue Cow - (303) 697-5721
6951 Collard's Espresso - (303) 972-6886
6952 Dream Cafe - (303) 697-1280
6953 Fort - (303) 697-9310
6954 High Point Restaurant Equipment - (303) 697-3735
6955 Mill Street Deli - (303) 697-1700
6956 Morrison Holiday Bar - (303) 697-5658
6957 Morrison Inn - (303) 697-6650
6958 Ranch Grill - (303) 697-4118
6959 Red Ro

In [7]:
for i in db.restaurant_atomic.find()[8925:8930]:
    print(i["name"], i["phone"])

Cosmo's Pizza (303) 447-3278
D'napoli Ristorante (303) 444-8434
Da Kind Pizza (303) 554-0500
Dagabi (303) 786-9004
Dairy Queen (303) 443-0656
